In [ ]:
#패키지 불러오기 및 각종 설치 & 폰트 설정 한방에 
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

# 폰트 설정 생략
# font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# font_name = fm.FontProperties(fname=font_path, size=10).get_name()
# print(font_name)
# plt.rc('font', family=font_name)

fm._rebuild()
matplotlib.rcParams['axes.unicode_minus'] = False

!pip install catboost
# !pip install pycaret==2.2.3
# !pip install numba==0.53
# !pip install numpy==1.17.3
from tqdm.notebook import tqdm
# from pycaret.regression import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import os

In [ ]:
# 파일 불러오기 (각자 경로 설정)
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/test.csv')
age_gender = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/age_gender_info.csv')

1위 코드의 결측치 처리

In [ ]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

1위 코드의 데이터 오류 부분 추가

In [ ]:
##1번 데이터 오류 (가장 큰 차이를 보이는 단지 4개에 대해 삭제 (토크 게시판에 크기순으로 나열되어있음))
train = train[train.단지코드 != 'C1804']
train = train[train.단지코드 != 'C2405']
train = train[train.단지코드 != 'C1740']
train = train[train.단지코드 != 'C1206']

##2번 데이터오류 
train = train[train.단지코드 != 'C2085']
train = train[train.단지코드 != 'C1397']
train = train[train.단지코드 != 'C2431']
train = train[train.단지코드 != 'C1649']
train = train[train.단지코드 != 'C1036']

##3번 데이터 오류 
train = train[train.단지코드 != 'C1095']
train = train[train.단지코드 != 'C2051']
train = train[train.단지코드 != 'C1218']
train = train[train.단지코드 != 'C1894']
train = train[train.단지코드 != 'C2483']
train = train[train.단지코드 != 'C1502']
train = train[train.단지코드 != 'C1988']

age_gender에서 미성년자(20대 미만) 비율만 가져와서 join

In [ ]:
minors = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)']
age_gender['미성년자'] = age_gender[minors].sum(axis=1)

In [ ]:
train = train.merge(age_gender, left_on= ["지역"], right_on= ["지역"], how='left')
test = test.merge(age_gender, left_on= ["지역"], right_on= ["지역"], how='left')

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
test.columns

성별 빼기

In [ ]:
train = train.drop(columns=['10대미만(여자)', '10대미만(남자)', '10대(여자)',
       '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)', '40대(여자)',
       '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)', '70대(여자)',
       '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)', '100대(여자)','100대(남자)'])


In [ ]:
test = test.drop(columns=['10대미만(여자)', '10대미만(남자)', '10대(여자)',
       '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)', '40대(여자)',
       '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)', '70대(여자)',
       '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)', '100대(여자)',
       '100대(남자)'])

In [ ]:
# 컬럼 위치 변경
train = train[['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '미성년자', '등록차량수']]

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
# 이름 변경
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '미성년자', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '미성년자'
]

지역 숫자로 매핑

In [ ]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

In [ ]:
train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

In [ ]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

전용면적 상/하한 적용

In [ ]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [ ]:
test['전용면적'].unique()

단지별 데이터 1차원으로 취합

In [ ]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

In [ ]:
new_train

결측치 처리

In [ ]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

학습

In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [ ]:
y_train

0       205.0
1      1064.0
2       730.0
3       553.0
4       415.0
        ...  
402      57.0
403     246.0
404      19.0
405      16.0
406     146.0
Name: 등록차량수, Length: 407, dtype: float64

In [ ]:
x_train.shape, y_train.shape, x_test.shape

((407, 21), (407,), (150, 21))

#**코드 생성_LSTM**

In [ ]:
x_shape = new_train.iloc[:, 1:-1].to_numpy()
y_shape = new_train.iloc[:,-1].to_numpy()
x_shape_test = new_test.iloc[:,1:].to_numpy()


In [ ]:
x_shape= x_shape.reshape(407,21,1)

In [ ]:
x_shape.shape, x_shape_test.shape

In [ ]:
#2. 모델구성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D

model = Sequential()
model.add(Conv1D(filters=256, kernel_size=1, padding='same',
                   strides=1 , input_shape=(21,1))) #x변경 됐음
model.add(Dense(64))
model.add(Dense(128))
model.add(Dense(32))
model.add(Dense(1)) 
model.summary()

In [ ]:
model.compile(loss='mse', optimizer= 'adam')
model.fit(x_shape,y_shape, epochs=10, batch_size=8)

In [ ]:
#4. 평가, 예측
loss=model.evaluate(x_shape,y_shape)
print(loss)

In [ ]:
x_shape_test= x_shape_test.reshape(150,21,1)

In [ ]:
reshape_pred = model.predict(x_shape_test)

In [ ]:
reshape_pred = reshape_pred.reshape(150, 21)

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['num'] = reshape_pred

#코드생성_RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

추론

In [ ]:
pred = model.predict(x_test)

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [ ]:
sample_submission['num'] = pred

In [ ]:
sample_submission.head()

,code,num
0,C1072,706.36
1,C1128,1218.16
2,C1456,529.05
3,C1840,562.21
4,C1332,1119.69


In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/221008_parking_data/221009_baseline.csv', index=False)